In [1]:

import pickle
import sys

from flask import Flask, request
from flask_restful import Resource, Api
from json import dumps
# from flask.ext.jsonpify import jsonify
from flask_jsonpify import jsonify
import pandas as pd
import numpy as np
import json




# Load Each Model

In [2]:
models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','SetDueDay']
modelpy_path = '../../../../Chatbot1.0/MLModel/code/{}/'
savedModel_path = '../../../../Chatbot1.0/MLModel/savedModel/{}/{}.pickle'
model_dict = {}
for each_model in models_list:
    sys.path.append(modelpy_path.format(each_model))
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))

In [3]:
model_dict['WillingToPay'].classify('我现在手头紧')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.663 seconds.
Prefix dict has been built succesfully.


(1, [1, 0.5564098042952339])

In [4]:
import pickle

class ClassifierBase:
    def load_model(self,**path):
        
        load_path = path.get('tfidf')
        if load_path is not None:
            self.tfidf = pickle.load(open(load_path, 'rb'))
            print('tfidf load successfully')
            
        load_path = path.get('svc')
        if load_path is not None:
            self.svc = pickle.load(open(load_path, 'rb'))
            print('svc load successfully')
            
        load_path = path.get('lightgbm')
        if load_path is not None:
            self.lightgbm = pickle.load(open(load_path, 'rb'))
            print('lightgbm load successfully')
            
        load_path = path.get('logistic')
        if load_path is not None:
            self.logistic = pickle.load(open(load_path, 'rb'))
            print('logistic load successfully')
            
            
####### classifier 0
####### InitClassifier

class InitClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to initialize conversation'
        self.label_explain = {0: 'next'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        
        return (0, 1)


    

    
    
####### classifier 0
####### InitClassifier

class OtherClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to handle irrelavant logic'
        self.label_explain = {0: 'next'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        
        return (0, 1)
    
class StopClassifier(ClassifierBase):
    def __init__(self, **model_path):
        self.load_model(**model_path)
        self.description = 'This model is used to handle stop logic'
        self.label_explain = {0: 'stop'}
        
    
            
            
    def classify(self,sentence):
        """
        0 - next
        """
        
        return (0, 1)
            

model_dict['OtherClassifier'] = OtherClassifier()
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()

In [5]:



class Node:
    def __init__(self, node_name, parent=None):
        self.name = node_name
        self.jump = None
        self.entry_counter = 0
        self.parent=parent
        self.ifConnected = False
  
        print('{} is initialized'.format(node_name))
        
        
    def summary(self):
        return {'node_name': self.name, 
                'jump_action':self.jump_action, 
                'description':self.describe, 
                'class_name':self.__class__.__name__, 
                'model': self.model_name}
    
    def _connect(self):
        pass
    
    def _triger_jump(self):
        pass
    
    def get_response(self):
        self.entry_counter += 1 
        self._connect()
        self._triger_jump()
        return self.response
    
    def process(self, sentence, model_dict):
        model = model_dict[self.model_name]
        _label, _confidence = model.classify(sentence)
        if self.connect is not None:
            self.output_label = _label
            self.output_confidence = _confidence
            return self.connect[_label], _label, _confidence
        else:
            return None,None,None
        
###################### Node 0  #########################



class NodeFunctional(Node):
    def __init__(self,node_name, parent=None):
        super().__init__(node_name,parent)
    
class S1_N0(NodeFunctional):
    def __init__(self,parent=None):
        super().__init__('s0',parent)
        self.describe = 'Init node'
        self.model_name = 'InitClassifier'
        self.response = 'S0 initialize'
        self.connect = {0: S1_N1(self)}
        
    
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N1(self)}
    
        
        
       
        
###################### Node 1  #########################
class S1_N1(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n1_identity_q',parent)
        self.describe = 'Verify Identify'
        self.model_name = 'IDClassifier'
        self.response = '你好，这里是H催收公司，请问是罗巍先生吗？'
                
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N15(self), 
                            1: S1_N5(self), 
                            2: S1_N4(self)}
        

    
        
#######################  Node 2  #############################        
class S1_N5(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n5_ifAcquainted_q',parent)
        self.describe = 'Ask if know debtor'
        self.model_name = 'IfKnowDebtor'
        self.response = '不好意思，打扰了，请问您认识罗先生吗！'
        
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N101(self), #know
                            1: S1_N102(self), #do not know
                            2: S1_N7(self)}
        
        
##########################  Node 3  ##########################        
class S1_N15(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n15_verifyWill_q',parent)
        self.describe = 'Verify willing to pay'
        self.model_name = 'WillingToPay'
        self.response = '你好，我是H催收公司的客服小催，您之前借贷了H公司5万块钱，约定在2018年5月1日还清。您已还款3万2千块钱，但因逾期未还产生了相应的利息和延迟还款费用，现在一共需要还2万块钱，其中1.2万本金，1500利息，500延迟还款手续费用，请问您打算什么时候处理下呢？'
       
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N20(self), #know
                            1: S1_N20(self), #do not know
                            2: S1_N25(self), # pay less
                            3: S1_N19(self) 
                           }
#########################  Node 5  ###########################        
class S1_N20(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n20_q4_setDue3Day',parent)
        self.describe = 'ask if can pay very soon'
        self.model_name = 'SetDueDay'
        self.response = '请问您能在3天之内还款吗？'
        
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N103(self), #know
                            1: self, #do not know
                            2: S1_N24(self), # pay less
                           }
            self.jump = {1: S1_N25(self)}
        
        
    def _triger_jump(self):
        if self.entry_counter >= 4:
            print('triger jump action')
            self.connect.update(self.jump)
            
    def get_response(self):
        self.entry_counter += 1 
        self._connect()
        self._triger_jump()
        if self.parent.output_label == 1:
            self.response_1 = '赖账你是赖不掉的，目前我们公司已经派专员处理了，现在要求您在3天以内还钱'
            return self.response_1
        return self.response
        
        
        
#########################  Node 7  ###########################        
class S1_N25(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n25_cutDebt_q',parent)
        self.describe = 'ask if accept less amount'
        self.model_name = 'CutDebt'
        self.response = '额... 如果是这样的话您看我帮您把金额减免一定程度可以吗？ 如果今天换，我帮您把利息1500全部剪掉，您只用还本金1万2。这是我能做大的最大程度了！'
        
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N33(self), #know
                            1: self, #do not know
                            2: S1_N30(self), # pay less
                           }
            self.jump = {1: S1_N32(self)}
        
    def _triger_jump(self):
        if self.entry_counter >= 4:
            print('triger jump action')
            self.connect.update(self.jump)
        
        
        
#########################  Node 8  ###########################        
class S1_N32(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n32_splitDebt_q',parent)
        self.describe = 'ask if accept installment'
        self.model_name = 'Installment'
        self.response = '那么您看这样行吗？ 您今天先还30%，也就是3600块钱，剩下的我为您申请下延期，但剩下的最晚需要在1个月内还清。您看可以吗？'
        
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N41(self), #yes
                            1: self, # no
                            2: S1_N35(self), 
                           }
            self.jump = {1: S1_N105(self)}
        
    def _triger_jump(self):
        if self.entry_counter >= 4:
            print('triger jump action')
            self.connect.update(self.jump)
        
        
        
#########################  Node 9  ###########################        
class S1_N33(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n33_setDue3Day',parent)
        self.describe = 'ask if can pay very soon'
        self.model_name = 'SetDueDay'
        self.response = '请问您能在3天之内还款吗？'
        
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N104(self), #yes
                            1: self, # no
                            2: S1_N40(self), 
                           }
            self.jump = {1: S1_N32(self)}
        
    def _triger_jump(self):
        if self.entry_counter >= 4:
            print('triger jump action')
            self.connect.update(self.jump)
        
        
        
#########################  Node 10  ###########################        
class S1_N41(NodeFunctional):
    def __init__(self,parent):
        super().__init__('cf_s1_n41_setSplitDebtDue_q',parent)
        self.describe = 'ask if can pay very soon'
        self.model_name = 'SetDueDay'
        self.response = '请问您能在3天之内还款吗？'
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.connect = {0: S1_N106(self), #yes
                            1: self, # no
                            2: S1_N44(self), 
                           }
            self.jump = {1: S1_N107(self)}
        
    def _triger_jump(self):
        if self.entry_counter >= 4:
            print('triger jump action')
            self.connect.update(self.jump)
        
        
        

        
#########################  Node Other ########################
class NodeOther(Node):
    def __init__(self,node_name=None, parent=None):
        super().__init__(node_name,parent)
        self.describe = 'other logic'
        self.model_name = 'OtherClassifier'
        self.response = '不好意思先生，您的回答我不太理解，请重复'
        
    def _connect(self):
        if not self.ifConnected:
            self.ifConnected = True
            self.model_name = self.parent.model_name
            self.connect = self.parent.connect
            self.process = self.parent.process
        
        



class S1_N4(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n4_identity_a_misc',parent)



        
class S1_N7(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n7_ifAcquainted_a_misc',parent)
        
    

        
class S1_N19(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n19_verifyWill_a_misc',parent)
        
        
    

class S1_N24(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n24_setDue_a_misc',parent)
        
        
class S1_N30(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n30_cutDebt_a_misc',parent)
        

class S1_N40(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n40_setCutDebtDue_a_misc',parent)
        
    
class S1_N35(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n35_splitDebt_a_misc',parent)
        
        
class S1_N44(NodeOther):
    def __init__(self,parent):
        super().__init__('cf_s1_n44_setSplitDebtDue_a_misc',parent)
        
        
        
############################## STOP NODE ########################
class NodeStop(Node):
    def __init__(self, parent=None):
        super().__init__(node_name,parent)
        self.model_name = 'StopClassifier'

        
    def _connect(self):
        self.connect = None



class S1_N101(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n101_ifAcquainted_s',parent)
        self.describe = 'inform phone recipient'
        self.response = '请您告诉xxx先生，请他尽快联系H催收公司，感谢您的配合，再见！'

        
        
class S1_N102(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n102_ifAcquainted_s',parent)
        self.describe = 'do not know debtor'
        self.response = '十分抱歉，打扰到您。感谢您的配合，再见！'
    
 
        
class S1_N103(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n103_paymentChannel_s',parent)
        self.describe = 'notify methods of paying'
        self.response = '感谢您的配合。您可以使用APP或者银行转账的方式进行还款，还请将还款截图发至微信hwchat！'
    

        
class S1_N104(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n104_paymentChannel_s',parent)
        self.describe = 'notify methods of paying'
        self.response = '感谢您的配合。您可以使用APP或者银行转账的方式进行还款，还请将还款截图发至微信hwchat！'
    
        
class S1_N105(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n105_noResult_s',parent)
        self.describe = 'no result'
        self.response = '您这也不行，那也不行！我们会近期再与您联系，到时候会是更强势的催收人员了！再见！'
        
    

class S1_N106(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n106_paymentChannel_s',parent)
        self.describe = 'notify methods of paying'
        self.response = '感谢您的配合。您可以使用APP或者银行转账的方式进行还款，还请将还款截图发至微信hwchat！'
        
        
class S1_N107(Node):
    def __init__(self,parent):
        super().__init__('cf_s1_n107_noResult_s',parent)
        self.describe = 'no result'
        self.response = '您这也不行，那也不行！我们会近期再与您联系，到时候会是更强势的催收人员了！再见！'
        
    
########################################################################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
######################################### Tree #########################################################################
      
        
class TreeBase:
    def __init__(self, node_pool, start_node='cf_s1_n1_identity_q', ):
        self.current_node_name = start_node
        self.node_pool = self.__get_nodePool(node_pool)
        
    def __get_nodePool(self, node_pool):
        diction = {}
        nodes_name = []
        for node in node_pool:
            cur = node()
            diction[cur.name] = cur
            nodes_name.append(cur.name)
        self.nodes_name = nodes_name
        return diction
    
class TreeStage1(TreeBase):
    def __init__(self, node_pool, start_node='s0'):
        super().__init__(node_pool, start_node='s0')
        self.build_graph()
        

        
    def classify(self, sentence, model_dict):
        current_node = self.node_pool[self.current_node_name]
        
        # classify
        print('Current node name: {}'.format(self.current_node_name))
        model = model_dict[current_node.model_name]
        label, confidence = model.classify(sentence)
        print('predict label is {} and confidence is {}'.format(label, confidence))
        
        # update current node
        self.current_node_name = self.graph[self.current_node_name][label]
        print('now go to label {}'.format(self.current_node_name))
        updated_node = self.node_pool[self.current_node_name]
        print(updated_node.response)
        
        return updated_node.response

In [6]:
cur = S1_N0()


s0 is initialized
cf_s1_n1_identity_q is initialized


In [7]:
cur,_l,_c = cur.process('我现在没钱',model_dict)
cur.get_response()

cf_s1_n15_verifyWill_q is initialized
cf_s1_n5_ifAcquainted_q is initialized
cf_s1_n4_identity_a_misc is initialized


'你好，这里是H催收公司，请问是罗巍先生吗？'

In [8]:
app = Flask(__name__)
api = Api(app)

# class ChatBot(Resource):
#     def get(self):
#         args = request.args
#         try:
#             return {'Chatbot': 'Hellow World! ' + args.get('data')}
#         except Exception as e:
#             print(e)
#             return {'Chatbot': 'Hellow World! ' }
    
class Cache:
    def __init__(self):
        self.session_pool = []
        self.active_session = {}
        
        
    def create_session(self, stage=1):
        available = list(set(range(1000)) - set(self.session_pool))
        if len(available) > 0:
            sessionId = available[0]
            self.session_pool.append(sessionId)
            if stage == 1:
                self.active_session[sessionId] = S1_N0()
        else:
            sessionId = None
        return sessionId
    
    
    
class CacheV1:
    def __init__(self):
        self.s1 = TreeStage1(node_pool)
        
    
            
            

    

class ChatBot(Resource):
    

    
    
    def get(self):
        args = request.args
        try:
            action = args.get('action')
            print(action)
        except Exception as e:
            return {'message': 'No action secified', 'status':'fail'}
        
        if action == 'create':
            sessionId = cache.create_session()
#             return {'test':'1'}
            
            return {'message':{'sessionId':sessionId}, 'status': 'successful'} 
        else:
            return {'message':'undefined action --- {}'.format(action)}
        
class ChatBotV1(Resource):
    

    
    
    def get(self):
        args = request.args
        action = args.get('action')
        if action == 'create':
            sessionId = cache.create_session()
            return {'message': {'sessionId':sessionId}, 
                    'status': 'successful'}
        elif action == 'chat':
            sessionId = args.get('sessionId')
            try:
                s = cache.active_session[int(sessionId)]
            except KeyError:
                return {'message': 'unknown session Id or session expired. Requested Session id is {}'.format(sessionId), 'status': 'failed'}
            data = args.get('data')
            cur_node, _label, _confidence = s.process(data, model_dict)
            message = cur_node.get_response()
            cache.active_session[int(sessionId)] = cur_node
            return {'message': message, 
                    'status': 'successful'}
        else:
            return {'message': 'Unknown Action! Received {} action'.format(action),
                    'status': 'failed'}
        
        
        
            
        
        
api.add_resource(ChatBotV1, '/chatbotv1')
cache = Cache()

In [ ]:
app.run(host='0.0.0.0', port='8889')

 * Running on http://0.0.0.0:8889/ (Press CTRL+C to quit)
172.18.0.1 - - [14/Jun/2018 20:57:55] "GET / HTTP/1.1" 404 -
172.18.0.1 - - [14/Jun/2018 20:57:55] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
hash(str(1))